# Technical Analysis: Indicators and Trading Signals

This notebook demonstrates various technical analysis indicators and how to generate trading signals.

## Topics Covered

1. Moving Averages (SMA, EMA)
2. MACD (Moving Average Convergence Divergence)
3. RSI (Relative Strength Index)
4. Bollinger Bands
5. Stochastic Oscillator
6. Volume Analysis
7. Support and Resistance Levels
8. Trading Signal Generation
9. Backtesting Simple Strategies

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('deep')

## 1. Fetch Stock Data

In [ ]:
# Define parameters
ticker = 'AAPL'
end_date = datetime.now()
start_date = end_date - timedelta(days=365)  # 1 year of data

# Download data
print(f"Fetching data for {ticker} from {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}")
data = yf.download(ticker, start=start_date, end=end_date)

# Display basic information
print(f"\nData shape: {data.shape}")
print(f"\nLast 5 days:")
data.tail()

## 2. Moving Averages

In [ ]:
# Calculate moving averages
data['SMA_20'] = data['Close'].rolling(window=20).mean()
data['SMA_50'] = data['Close'].rolling(window=50).mean()
data['SMA_200'] = data['Close'].rolling(window=200).mean()

# Calculate exponential moving averages
data['EMA_12'] = data['Close'].ewm(span=12, adjust=False).mean()
data['EMA_26'] = data['Close'].ewm(span=26, adjust=False).mean()

# Plotting
fig, ax = plt.subplots(figsize=(14, 8))

ax.plot(data.index, data['Close'], label='Close Price', linewidth=2, color='black')
ax.plot(data.index, data['SMA_20'], label='SMA 20', alpha=0.7)
ax.plot(data.index, data['SMA_50'], label='SMA 50', alpha=0.7)
ax.plot(data.index, data['SMA_200'], label='SMA 200', alpha=0.7)

ax.set_title(f'{ticker} - Moving Averages', fontsize=16)
ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Price ($)', fontsize=12)
ax.legend(loc='best')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 3. MACD (Moving Average Convergence Divergence)

In [ ]:
# Calculate MACD
data['MACD'] = data['EMA_12'] - data['EMA_26']
data['MACD_signal'] = data['MACD'].ewm(span=9, adjust=False).mean()
data['MACD_histogram'] = data['MACD'] - data['MACD_signal']

# Plotting
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10), sharex=True)

# Price chart
ax1.plot(data.index, data['Close'], label='Close Price', linewidth=2)
ax1.set_ylabel('Price ($)', fontsize=12)
ax1.set_title(f'{ticker} - MACD Analysis', fontsize=16)
ax1.grid(True, alpha=0.3)

# MACD chart
ax2.plot(data.index, data['MACD'], label='MACD', color='blue', linewidth=2)
ax2.plot(data.index, data['MACD_signal'], label='Signal Line', color='red', linewidth=2)
ax2.bar(data.index, data['MACD_histogram'], label='MACD Histogram', alpha=0.3, color='gray')
ax2.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
ax2.set_xlabel('Date', fontsize=12)
ax2.set_ylabel('MACD', fontsize=12)
ax2.legend(loc='best')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. RSI (Relative Strength Index)

In [ ]:
def calculate_rsi(data, period=14):
    """Calculate Relative Strength Index"""
    delta = data.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

# Calculate RSI
data['RSI'] = calculate_rsi(data['Close'])

# Plotting
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10), sharex=True)

# Price chart
ax1.plot(data.index, data['Close'], label='Close Price', linewidth=2)
ax1.set_ylabel('Price ($)', fontsize=12)
ax1.set_title(f'{ticker} - RSI Analysis', fontsize=16)
ax1.grid(True, alpha=0.3)

# RSI chart
ax2.plot(data.index, data['RSI'], label='RSI', color='purple', linewidth=2)
ax2.axhline(y=70, color='r', linestyle='--', alpha=0.7, label='Overbought (70)')
ax2.axhline(y=30, color='g', linestyle='--', alpha=0.7, label='Oversold (30)')
ax2.fill_between(data.index, 30, 70, alpha=0.1, color='gray')
ax2.set_xlabel('Date', fontsize=12)
ax2.set_ylabel('RSI', fontsize=12)
ax2.set_ylim(0, 100)
ax2.legend(loc='best')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Bollinger Bands

In [ ]:
# Calculate Bollinger Bands
period = 20
num_std = 2

data['BB_Middle'] = data['Close'].rolling(window=period).mean()
bb_std = data['Close'].rolling(window=period).std()
data['BB_Upper'] = data['BB_Middle'] + (bb_std * num_std)
data['BB_Lower'] = data['BB_Middle'] - (bb_std * num_std)
data['BB_Width'] = data['BB_Upper'] - data['BB_Lower']
data['BB_Position'] = (data['Close'] - data['BB_Lower']) / (data['BB_Upper'] - data['BB_Lower'])

# Plotting
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10), sharex=True)

# Price with Bollinger Bands
ax1.plot(data.index, data['Close'], label='Close Price', linewidth=2, color='black')
ax1.plot(data.index, data['BB_Upper'], label='Upper Band', alpha=0.7, color='red')
ax1.plot(data.index, data['BB_Middle'], label='Middle Band (SMA 20)', alpha=0.7, color='blue')
ax1.plot(data.index, data['BB_Lower'], label='Lower Band', alpha=0.7, color='green')
ax1.fill_between(data.index, data['BB_Upper'], data['BB_Lower'], alpha=0.1, color='gray')
ax1.set_ylabel('Price ($)', fontsize=12)
ax1.set_title(f'{ticker} - Bollinger Bands', fontsize=16)
ax1.legend(loc='best')
ax1.grid(True, alpha=0.3)

# Bollinger Band Width
ax2.plot(data.index, data['BB_Width'], label='Band Width', color='orange', linewidth=2)
ax2.set_xlabel('Date', fontsize=12)
ax2.set_ylabel('Band Width', fontsize=12)
ax2.set_title('Bollinger Band Width (Volatility)', fontsize=14)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Stochastic Oscillator

In [ ]:
# Calculate Stochastic Oscillator
period = 14
k_period = 3
d_period = 3

# Calculate %K
low_min = data['Low'].rolling(window=period).min()
high_max = data['High'].rolling(window=period).max()
data['Stoch_K'] = 100 * ((data['Close'] - low_min) / (high_max - low_min))
data['Stoch_K'] = data['Stoch_K'].rolling(window=k_period).mean()

# Calculate %D
data['Stoch_D'] = data['Stoch_K'].rolling(window=d_period).mean()

# Plotting
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10), sharex=True)

# Price chart
ax1.plot(data.index, data['Close'], label='Close Price', linewidth=2)
ax1.set_ylabel('Price ($)', fontsize=12)
ax1.set_title(f'{ticker} - Stochastic Oscillator', fontsize=16)
ax1.grid(True, alpha=0.3)

# Stochastic Oscillator
ax2.plot(data.index, data['Stoch_K'], label='%K', color='blue', linewidth=2)
ax2.plot(data.index, data['Stoch_D'], label='%D', color='red', linewidth=2)
ax2.axhline(y=80, color='r', linestyle='--', alpha=0.7, label='Overbought (80)')
ax2.axhline(y=20, color='g', linestyle='--', alpha=0.7, label='Oversold (20)')
ax2.fill_between(data.index, 20, 80, alpha=0.1, color='gray')
ax2.set_xlabel('Date', fontsize=12)
ax2.set_ylabel('Stochastic %', fontsize=12)
ax2.set_ylim(0, 100)
ax2.legend(loc='best')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 7. Volume Analysis

In [ ]:
# Calculate volume indicators
data['Volume_SMA'] = data['Volume'].rolling(window=20).mean()
data['Volume_Ratio'] = data['Volume'] / data['Volume_SMA']

# On-Balance Volume (OBV)
data['OBV'] = (np.sign(data['Close'].diff()) * data['Volume']).fillna(0).cumsum()

# Plotting
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(14, 12), sharex=True)

# Price chart
ax1.plot(data.index, data['Close'], label='Close Price', linewidth=2)
ax1.set_ylabel('Price ($)', fontsize=12)
ax1.set_title(f'{ticker} - Volume Analysis', fontsize=16)
ax1.grid(True, alpha=0.3)

# Volume chart
colors = ['g' if data['Close'].iloc[i] >= data['Open'].iloc[i] else 'r' 
          for i in range(len(data))]
ax2.bar(data.index, data['Volume'], color=colors, alpha=0.7, label='Volume')
ax2.plot(data.index, data['Volume_SMA'], color='blue', linewidth=2, label='Volume SMA(20)')
ax2.set_ylabel('Volume', fontsize=12)
ax2.legend(loc='best')
ax2.grid(True, alpha=0.3)

# OBV chart
ax3.plot(data.index, data['OBV'], label='On-Balance Volume', color='purple', linewidth=2)
ax3.set_xlabel('Date', fontsize=12)
ax3.set_ylabel('OBV', fontsize=12)
ax3.legend(loc='best')
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 8. Support and Resistance Levels

In [ ]:
# Find support and resistance levels using pivot points
def find_pivot_points(data, window=20):
    """Find local minima and maxima as support and resistance"""
    highs = data['High'].rolling(window=window, center=True).max()
    lows = data['Low'].rolling(window=window, center=True).min()
    
    # Find where current high/low equals rolling max/min
    resistance_points = data[data['High'] == highs]['High']
    support_points = data[data['Low'] == lows]['Low']
    
    return support_points, resistance_points

support_levels, resistance_levels = find_pivot_points(data)

# Calculate key levels using recent data
recent_data = data.tail(60)  # Last 60 days
key_resistance = recent_data['High'].nlargest(3).mean()
key_support = recent_data['Low'].nsmallest(3).mean()

# Plotting
fig, ax = plt.subplots(figsize=(14, 8))

# Candlestick-style plot
for idx, row in data.iterrows():
    color = 'g' if row['Close'] >= row['Open'] else 'r'
    ax.plot([idx, idx], [row['Low'], row['High']], color=color, linewidth=1)
    ax.plot([idx, idx], [row['Open'], row['Close']], color=color, linewidth=3)

# Plot support and resistance levels
ax.axhline(y=key_resistance, color='red', linestyle='--', alpha=0.7, 
          label=f'Key Resistance: ${key_resistance:.2f}')
ax.axhline(y=key_support, color='green', linestyle='--', alpha=0.7, 
          label=f'Key Support: ${key_support:.2f}')

# Mark pivot points
for date, price in resistance_levels.items():
    ax.scatter(date, price, color='red', s=50, marker='v', alpha=0.7)
for date, price in support_levels.items():
    ax.scatter(date, price, color='green', s=50, marker='^', alpha=0.7)

ax.set_title(f'{ticker} - Support and Resistance Levels', fontsize=16)
ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Price ($)', fontsize=12)
ax.legend(loc='best')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 9. Trading Signal Generation

In [ ]:
# Generate trading signals based on multiple indicators
def generate_signals(data):
    """Generate buy/sell signals based on technical indicators"""
    signals = pd.DataFrame(index=data.index)
    signals['price'] = data['Close']
    
    # Moving Average Crossover
    signals['ma_signal'] = 0
    signals.loc[data['SMA_20'] > data['SMA_50'], 'ma_signal'] = 1
    signals.loc[data['SMA_20'] < data['SMA_50'], 'ma_signal'] = -1
    
    # MACD Signal
    signals['macd_signal'] = 0
    signals.loc[data['MACD'] > data['MACD_signal'], 'macd_signal'] = 1
    signals.loc[data['MACD'] < data['MACD_signal'], 'macd_signal'] = -1
    
    # RSI Signal
    signals['rsi_signal'] = 0
    signals.loc[data['RSI'] < 30, 'rsi_signal'] = 1  # Oversold
    signals.loc[data['RSI'] > 70, 'rsi_signal'] = -1  # Overbought
    
    # Bollinger Bands Signal
    signals['bb_signal'] = 0
    signals.loc[data['Close'] < data['BB_Lower'], 'bb_signal'] = 1
    signals.loc[data['Close'] > data['BB_Upper'], 'bb_signal'] = -1
    
    # Combined Signal (majority vote)
    signals['combined_score'] = (signals['ma_signal'] + signals['macd_signal'] + 
                                signals['rsi_signal'] + signals['bb_signal'])
    signals['position'] = 0
    signals.loc[signals['combined_score'] >= 2, 'position'] = 1  # Buy
    signals.loc[signals['combined_score'] <= -2, 'position'] = -1  # Sell
    
    # Generate actual trading signals (changes in position)
    signals['signal'] = signals['position'].diff()
    
    return signals

# Generate signals
signals = generate_signals(data)

# Extract buy and sell signals
buy_signals = signals[signals['signal'] > 0]
sell_signals = signals[signals['signal'] < 0]

print(f"Total Buy Signals: {len(buy_signals)}")
print(f"Total Sell Signals: {len(sell_signals)}")
print(f"\nLast 5 Buy Signals:")
print(buy_signals[['price', 'combined_score']].tail())

## 10. Trading Signals Visualization

In [ ]:
# Create comprehensive trading signal chart
fig, axes = plt.subplots(5, 1, figsize=(14, 16), sharex=True)

# 1. Price with signals
axes[0].plot(data.index, data['Close'], label='Close Price', linewidth=2, color='black')
axes[0].scatter(buy_signals.index, buy_signals['price'], color='green', 
               marker='^', s=100, label='Buy Signal', zorder=5)
axes[0].scatter(sell_signals.index, sell_signals['price'], color='red', 
               marker='v', s=100, label='Sell Signal', zorder=5)
axes[0].set_ylabel('Price ($)', fontsize=12)
axes[0].set_title(f'{ticker} - Trading Signals Dashboard', fontsize=16)
axes[0].legend(loc='best')
axes[0].grid(True, alpha=0.3)

# 2. MACD
axes[1].plot(data.index, data['MACD'], label='MACD', color='blue')
axes[1].plot(data.index, data['MACD_signal'], label='Signal', color='red')
axes[1].bar(data.index, data['MACD_histogram'], label='Histogram', alpha=0.3)
axes[1].axhline(y=0, color='black', linestyle='-', linewidth=0.5)
axes[1].set_ylabel('MACD', fontsize=12)
axes[1].legend(loc='best', fontsize=10)
axes[1].grid(True, alpha=0.3)

# 3. RSI
axes[2].plot(data.index, data['RSI'], label='RSI', color='purple')
axes[2].axhline(y=70, color='r', linestyle='--', alpha=0.5)
axes[2].axhline(y=30, color='g', linestyle='--', alpha=0.5)
axes[2].fill_between(data.index, 30, 70, alpha=0.1)
axes[2].set_ylabel('RSI', fontsize=12)
axes[2].set_ylim(0, 100)
axes[2].grid(True, alpha=0.3)

# 4. Bollinger Bands Position
axes[3].plot(data.index, data['BB_Position'], label='BB Position', color='orange')
axes[3].axhline(y=1, color='r', linestyle='--', alpha=0.5, label='Upper Band')
axes[3].axhline(y=0, color='g', linestyle='--', alpha=0.5, label='Lower Band')
axes[3].fill_between(data.index, 0, 1, alpha=0.1)
axes[3].set_ylabel('BB Position', fontsize=12)
axes[3].legend(loc='best', fontsize=10)
axes[3].grid(True, alpha=0.3)

# 5. Combined Signal Score
axes[4].bar(signals.index, signals['combined_score'], 
           color=['green' if x > 0 else 'red' for x in signals['combined_score']], 
           alpha=0.7)
axes[4].axhline(y=2, color='g', linestyle='--', label='Buy Threshold')
axes[4].axhline(y=-2, color='r', linestyle='--', label='Sell Threshold')
axes[4].set_xlabel('Date', fontsize=12)
axes[4].set_ylabel('Signal Score', fontsize=12)
axes[4].legend(loc='best', fontsize=10)
axes[4].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 11. Simple Strategy Backtesting

In [ ]:
# Backtest the combined signal strategy
def backtest_strategy(data, signals, initial_capital=10000):
    """Simple backtesting of the trading strategy"""
    positions = pd.DataFrame(index=signals.index)
    positions['shares'] = 100 * signals['position']  # 100 shares per signal
    
    # Calculate portfolio value
    portfolio = pd.DataFrame(index=signals.index)
    portfolio['positions'] = positions['shares'] * data['Close']
    portfolio['cash'] = initial_capital - (positions['shares'].diff().fillna(0) * data['Close']).cumsum()
    portfolio['total'] = portfolio['positions'] + portfolio['cash']
    portfolio['returns'] = portfolio['total'].pct_change()
    
    # Calculate buy and hold strategy
    buy_hold = pd.DataFrame(index=signals.index)
    initial_shares = initial_capital / data['Close'].iloc[0]
    buy_hold['total'] = initial_shares * data['Close']
    buy_hold['returns'] = buy_hold['total'].pct_change()
    
    return portfolio, buy_hold

# Run backtest
portfolio, buy_hold = backtest_strategy(data, signals)

# Calculate performance metrics
strategy_return = (portfolio['total'].iloc[-1] / portfolio['total'].iloc[0] - 1) * 100
buy_hold_return = (buy_hold['total'].iloc[-1] / buy_hold['total'].iloc[0] - 1) * 100
strategy_sharpe = np.sqrt(252) * portfolio['returns'].mean() / portfolio['returns'].std()
buy_hold_sharpe = np.sqrt(252) * buy_hold['returns'].mean() / buy_hold['returns'].std()

# Plotting
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))

# Portfolio value comparison
ax1.plot(portfolio.index, portfolio['total'], label='Strategy', linewidth=2)
ax1.plot(buy_hold.index, buy_hold['total'], label='Buy & Hold', linewidth=2)
ax1.set_ylabel('Portfolio Value ($)', fontsize=12)
ax1.set_title('Strategy Performance vs Buy & Hold', fontsize=16)
ax1.legend(loc='best')
ax1.grid(True, alpha=0.3)

# Drawdown comparison
strategy_dd = (portfolio['total'] / portfolio['total'].expanding().max() - 1) * 100
buyhold_dd = (buy_hold['total'] / buy_hold['total'].expanding().max() - 1) * 100

ax2.fill_between(strategy_dd.index, 0, strategy_dd, alpha=0.3, color='red', label='Strategy Drawdown')
ax2.fill_between(buyhold_dd.index, 0, buyhold_dd, alpha=0.3, color='blue', label='Buy & Hold Drawdown')
ax2.set_xlabel('Date', fontsize=12)
ax2.set_ylabel('Drawdown (%)', fontsize=12)
ax2.legend(loc='best')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print performance summary
print("\nPerformance Summary:")
print("="*50)
print(f"Strategy Return: {strategy_return:.2f}%")
print(f"Buy & Hold Return: {buy_hold_return:.2f}%")
print(f"Strategy Sharpe Ratio: {strategy_sharpe:.3f}")
print(f"Buy & Hold Sharpe Ratio: {buy_hold_sharpe:.3f}")
print(f"Max Strategy Drawdown: {strategy_dd.min():.2f}%")
print(f"Max Buy & Hold Drawdown: {buyhold_dd.min():.2f}%")
print(f"Number of Trades: {len(buy_signals) + len(sell_signals)}")

## Summary and Key Insights

This notebook demonstrated a comprehensive technical analysis workflow:

### 1. **Technical Indicators**
   - Moving Averages (SMA, EMA) for trend identification
   - MACD for momentum and trend changes
   - RSI for overbought/oversold conditions
   - Bollinger Bands for volatility and mean reversion
   - Stochastic Oscillator for momentum

### 2. **Volume Analysis**
   - Volume patterns confirm price movements
   - On-Balance Volume (OBV) shows buying/selling pressure

### 3. **Support and Resistance**
   - Key price levels for entry/exit decisions
   - Pivot points identify potential reversals

### 4. **Signal Generation**
   - Combined multiple indicators for robust signals
   - Avoided reliance on single indicator

### 5. **Backtesting Results**
   - Performance comparison with buy-and-hold strategy
   - Risk-adjusted returns through Sharpe ratio
   - Drawdown analysis for risk assessment

### Important Considerations:
- Past performance doesn't guarantee future results
- Transaction costs and slippage not included
- Market conditions change; strategies need adaptation
- Risk management is crucial for real trading
- Consider fundamental analysis alongside technical analysis